# 前言

1. 先做简单的“数据评估”—用df.head()看前几行数据结构、df.info()查看数据类型和缺失值分布、df.describe()了解数值型数据的统计特征（如是否有异常值），这一步能帮你精准定位“脏数据”在哪，避免后续清洗盲目操作。
2. 读取文件（如CSV，Excel,json）生成DataFrame（由多个Series组成的二维表）,通过pandas库中提供的方法对df/s数据从数据结构，内容方面进行清理
3. 最后，写入文件保存前，建议增加“清洗验证”环节。避免清洗后仍有遗漏：比如用df.isnull().sum()再次检查是否还有未处理的缺失值，用df.duplicated().sum()确认重复值已清除，用df.dtypes验证数据格式是否符合预期。确认无误后，再用df.to_csv()或df.to_excel()写入文件，形成“评估→清洗→验证→保存”的完整闭环。



1. [Pandas 官方文档主页](https://pandas.pydata.org/docs/reference/)
2. [Series 方法相关文档](https://pandas.pydata.org/docs/reference/api/pandas.Series.html)
3. [DataFrame 方法相关文档](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)


pandas库下的一些默认操作：<br>
axis=0 是默认值对每一列进行操作<br>大多数Pandas方法的inplace参数默认为False即会返回一个新的对象，而不修改原始数据<br>在去重复值中keep='first' 即默认保留第一次出现的重复项<br>df.sum()默认跳过NaN值进行计算<br>df.to_csv("file.csv")默认会保存索引列从而导致再读取的时候会把该列当作属性列处理这就需要我们写入时df.to_csv("file.csv", index=False)手动确保不默认保存标签列

In [1]:
import pandas as pd

# 清洗索引与列名

## 对索引和列名重命名（使用字典）

在读取数据后若发现数据的索引亦或是列名乱七八糟则可以用dataframe的rename方法
- 改索引：df1.rename(index={})
- 改列名: df1.rename(columns={})

In [7]:
df1 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example1.csv")
df1

,Unnamed: 0,客户_姓名,客户 性别,age,邮箱
0,1,李十,女,58,yVaQXekW@example.com
1,2_,冯三,男,27,LctXbrEM@example.com
2,3,吴十,男,32,UyTYBzUZ@example.com
3,4,王十,女,58,KCixgciF@example.com
4,_5,钱十,女,27,nqRBXOtA@example.com
5,6*,赵八,女,51,GyKwILAL@example.com


*在对json文件与csv文件使用read函数时候，当文件中没有表头的时候要加入可选参数header = None可以让pandas不以第一行数据作为表头当你不想用默认的下标012来索引每一行数据可以加入可选参数index_col=0 <br>意思是：把 CSV 文件里的第一列（第0列）当作 DataFrame 的行索引（行标签），而不是当作普通的数据列。*

In [8]:
df2 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example1.csv",index_col=0)
df2

,客户_姓名,客户 性别,age,邮箱
1,李十,女,58,yVaQXekW@example.com
2_,冯三,男,27,LctXbrEM@example.com
3,吴十,男,32,UyTYBzUZ@example.com
4,王十,女,58,KCixgciF@example.com
_5,钱十,女,27,nqRBXOtA@example.com
6*,赵八,女,51,GyKwILAL@example.com


In [9]:
df2.rename(index={'2_':'2','_5':5,'6*':6})

,客户_姓名,客户 性别,age,邮箱
1,李十,女,58,yVaQXekW@example.com
2,冯三,男,27,LctXbrEM@example.com
3,吴十,男,32,UyTYBzUZ@example.com
4,王十,女,58,KCixgciF@example.com
5,钱十,女,27,nqRBXOtA@example.com
6,赵八,女,51,GyKwILAL@example.com


In [10]:
df2

,客户_姓名,客户 性别,age,邮箱
1,李十,女,58,yVaQXekW@example.com
2_,冯三,男,27,LctXbrEM@example.com
3,吴十,男,32,UyTYBzUZ@example.com
4,王十,女,58,KCixgciF@example.com
_5,钱十,女,27,nqRBXOtA@example.com
6*,赵八,女,51,GyKwILAL@example.com


**此时你会发现并不改变原先df的值，此时你可用inplace=True或者对df2再赋值方法实现改变原df中数据（只改变df不变原文件）**

In [14]:
df2.rename(index={'2_':'2','_5':5,'6*':6},inplace=True)
df2

,客户_姓名,客户 性别,age,邮箱
1,李十,女,58,yVaQXekW@example.com
2,冯三,男,27,LctXbrEM@example.com
3,吴十,男,32,UyTYBzUZ@example.com
4,王十,女,58,KCixgciF@example.com
5,钱十,女,27,nqRBXOtA@example.com
6,赵八,女,51,GyKwILAL@example.com


In [15]:
df2 = df2.rename(columns={'客户_姓名':'客户姓名','客户 性别':'客户性别','age':'客户年龄','邮箱':'客户邮箱'})
df2

,客户姓名,客户性别,客户年龄,客户邮箱
1,李十,女,58,yVaQXekW@example.com
2,冯三,男,27,LctXbrEM@example.com
3,吴十,男,32,UyTYBzUZ@example.com
4,王十,女,58,KCixgciF@example.com
5,钱十,女,27,nqRBXOtA@example.com
6,赵八,女,51,GyKwILAL@example.com


## 对索引和列名重命名（使用方法/函数）-可以是pandas自带的也可以自定义

df.rename(index=函数/方法)
df.rename(columns=函数/方法)

In [16]:
s1 = pd.Series(["hello", "this", "is", "Lin"])
s1

0    hello
1     this
2       is
3      Lin
dtype: object

In [17]:
s1.str

In [18]:
s1.str.upper()

0    HELLO
1     THIS
2       IS
3      LIN
dtype: object

In [19]:
df3 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example2.csv")
df3

,Date,Amount,Salesperson,Location
0,2022-01-01,1000,Alice,New York
1,2022-01-02,1500,Bob,San Francisco
2,2022-01-03,800,Charlie,New York
3,2022-01-04,1200,David,San Francisco


In [24]:
df3.rename(columns=str.upper,inplace=True)
df3

,DATE,AMOUNT,SALESPERSON,LOCATION
0,2022-01-01,1000,Alice,New York
1,2022-01-02,1500,Bob,San Francisco
2,2022-01-03,800,Charlie,New York
3,2022-01-04,1200,David,San Francisco


## 把某列设为索引

In [25]:
df3

,DATE,AMOUNT,SALESPERSON,LOCATION
0,2022-01-01,1000,Alice,New York
1,2022-01-02,1500,Bob,San Francisco
2,2022-01-03,800,Charlie,New York
3,2022-01-04,1200,David,San Francisco


In [29]:
df4 = df3.set_index(str.upper("Salesperson"))
df4

,DATE,AMOUNT,LOCATION
SALESPERSON,,,
Alice,2022-01-01,1000,New York
Bob,2022-01-02,1500,San Francisco
Charlie,2022-01-03,800,New York
David,2022-01-04,1200,San Francisco


## 重设索引

In [31]:
df4.reset_index()

,SALESPERSON,DATE,AMOUNT,LOCATION
0,Alice,2022-01-01,1000,New York
1,Bob,2022-01-02,1500,San Francisco
2,Charlie,2022-01-03,800,New York
3,David,2022-01-04,1200,San Francisco


## 对索引和列名重新排序

In [36]:
df5 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example3.csv")
df5

,Unnamed: 0,客户姓名,客户性别,客户年龄,客户邮箱
0,a,李十,女,58,yVaQXekW@example.com
1,c,冯三,男,27,LctXbrEM@example.com
2,b,吴十,男,32,UyTYBzUZ@example.com
3,e,王十,女,58,KCixgciF@example.com
4,d,钱十,女,27,nqRBXOtA@example.com
5,f,赵八,女,51,GyKwILAL@example.com


**原数据中已用标签abc进行索引但是由于pandas会把数据集中每一列均当作属性处理并用默认012去索引所以此时会产生以上情况，
原为标签索引的列abc转为df后被当作没有属性名的一列此时需要用index=0来修改**

In [38]:
df5 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example3.csv",index_col=0)
df5

,客户姓名,客户性别,客户年龄,客户邮箱
a,李十,女,58,yVaQXekW@example.com
c,冯三,男,27,LctXbrEM@example.com
b,吴十,男,32,UyTYBzUZ@example.com
e,王十,女,58,KCixgciF@example.com
d,钱十,女,27,nqRBXOtA@example.com
f,赵八,女,51,GyKwILAL@example.com


In [41]:
df5.sort_index(inplace=True)
df5

,客户姓名,客户性别,客户年龄,客户邮箱
a,李十,女,58,yVaQXekW@example.com
b,吴十,男,32,UyTYBzUZ@example.com
c,冯三,男,27,LctXbrEM@example.com
d,钱十,女,27,nqRBXOtA@example.com
e,王十,女,58,KCixgciF@example.com
f,赵八,女,51,GyKwILAL@example.com


In [42]:
df5.sort_index(axis=1)

,客户姓名,客户年龄,客户性别,客户邮箱
a,李十,58,女,yVaQXekW@example.com
b,吴十,32,男,UyTYBzUZ@example.com
c,冯三,27,男,LctXbrEM@example.com
d,钱十,27,女,nqRBXOtA@example.com
e,王十,58,女,KCixgciF@example.com
f,赵八,51,女,GyKwILAL@example.com


In [43]:
df5.sort_values(by='客户年龄')

,客户姓名,客户性别,客户年龄,客户邮箱
d,钱十,女,27,nqRBXOtA@example.com
c,冯三,男,27,LctXbrEM@example.com
b,吴十,男,32,UyTYBzUZ@example.com
f,赵八,女,51,GyKwILAL@example.com
a,李十,女,58,yVaQXekW@example.com
e,王十,女,58,KCixgciF@example.com


In [44]:
df5.sort_values(by='客户年龄', ascending=False)

,客户姓名,客户性别,客户年龄,客户邮箱
a,李十,女,58,yVaQXekW@example.com
e,王十,女,58,KCixgciF@example.com
f,赵八,女,51,GyKwILAL@example.com
b,吴十,男,32,UyTYBzUZ@example.com
d,钱十,女,27,nqRBXOtA@example.com
c,冯三,男,27,LctXbrEM@example.com


In [46]:
df5.sort_values(by=['客户性别', '客户年龄'])

,客户姓名,客户性别,客户年龄,客户邮箱
d,钱十,女,27,nqRBXOtA@example.com
f,赵八,女,51,GyKwILAL@example.com
a,李十,女,58,yVaQXekW@example.com
e,王十,女,58,KCixgciF@example.com
c,冯三,男,27,LctXbrEM@example.com
b,吴十,男,32,UyTYBzUZ@example.com


*先按性别后按年龄<br>df.sort_values(by='列名', na_position='first'默认升序且将NaN放前面<br>列1升序，列2降序*

In [47]:
df5.sort_values(by=['客户性别', '客户年龄'], ascending=[True, False])

,客户姓名,客户性别,客户年龄,客户邮箱
a,李十,女,58,yVaQXekW@example.com
e,王十,女,58,KCixgciF@example.com
f,赵八,女,51,GyKwILAL@example.com
d,钱十,女,27,nqRBXOtA@example.com
b,吴十,男,32,UyTYBzUZ@example.com
c,冯三,男,27,LctXbrEM@example.com


In [48]:
df5

,客户姓名,客户性别,客户年龄,客户邮箱
a,李十,女,58,yVaQXekW@example.com
b,吴十,男,32,UyTYBzUZ@example.com
c,冯三,男,27,LctXbrEM@example.com
d,钱十,女,27,nqRBXOtA@example.com
e,王十,女,58,KCixgciF@example.com
f,赵八,女,51,GyKwILAL@example.com


In [49]:
df5.sort_values(by=['客户性别', '客户年龄'],inplace=True)
df5

,客户姓名,客户性别,客户年龄,客户邮箱
d,钱十,女,27,nqRBXOtA@example.com
f,赵八,女,51,GyKwILAL@example.com
a,李十,女,58,yVaQXekW@example.com
e,王十,女,58,KCixgciF@example.com
c,冯三,男,27,LctXbrEM@example.com
b,吴十,男,32,UyTYBzUZ@example.com


# 对数据进行转置

In [2]:
import pandas as pd

In [4]:
df6 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example4.csv", index_col=0)
df6 

,0,1,2
姓名,张三,李四,王五
年龄,30,25,28
地址,北京,上海,广州


In [5]:
df6 = df6.T
df6

,姓名,年龄,地址
0,张三,30,北京
1,李四,25,上海
2,王五,28,广州


# 对列进行拆分

In [7]:
df7 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example5.csv")
df7

,城市,人口密度
0,城市A,28.53万人/0.87平方公里
1,城市B,37.83万人/2.19平方公里
2,城市C,15.3万人/1.57平方公里


In [8]:
"28.53万人/0.87平方公里".split("/")

['28.53万人', '0.87平方公里']

In [9]:
"28.53万人和0.87平方公里".split("和")

['28.53万人', '0.87平方公里']

In [10]:
df7["人口密度"]

0    28.53万人/0.87平方公里
1    37.83万人/2.19平方公里
2     15.3万人/1.57平方公里
Name: 人口密度, dtype: object

In [12]:
df7["人口密度"].str.split("/")

0    [28.53万人, 0.87平方公里]
1    [37.83万人, 2.19平方公里]
2     [15.3万人, 1.57平方公里]
Name: 人口密度, dtype: object

In [13]:
type(df7["人口密度"].str.split("/"))

pandas.core.series.Series

In [14]:
df7["人口密度"].str.split("/", expand=True)

,0,1
0,28.53万人,0.87平方公里
1,37.83万人,2.19平方公里
2,15.3万人,1.57平方公里


In [15]:
type(df7["人口密度"].str.split("/", expand=True))

pandas.core.frame.DataFrame

In [16]:
df7

,城市,人口密度
0,城市A,28.53万人/0.87平方公里
1,城市B,37.83万人/2.19平方公里
2,城市C,15.3万人/1.57平方公里


In [17]:
df7[["人口", "面积"]] = df7["人口密度"].str.split("/", expand=True)
df7

,城市,人口密度,人口,面积
0,城市A,28.53万人/0.87平方公里,28.53万人,0.87平方公里
1,城市B,37.83万人/2.19平方公里,37.83万人,2.19平方公里
2,城市C,15.3万人/1.57平方公里,15.3万人,1.57平方公里


**对不存在的列名赋值就是再创建新列<br>很多情况函数都默认axis=o(对↓方向进行操作)<br>很多情况下函数都默认不操作原df只显示操作后的结果<br>这需要我们手动传参**

In [18]:
df7 = df7.drop("人口密度", axis=1)
df7

,城市,人口,面积
0,城市A,28.53万人,0.87平方公里
1,城市B,37.83万人,2.19平方公里
2,城市C,15.3万人,1.57平方公里


# 把不同列合并成一列

In [49]:
df8 = pd.DataFrame({'姓': ['张', '李', '王'], '名': ['三', '四', '五'], '年龄': [30, 25, 35]})
df8

,姓,名,年龄
0,张,三,30
1,李,四,25
2,王,五,35


In [50]:
df8["姓"]

0    张
1    李
2    王
Name: 姓, dtype: object

In [51]:
df8["姓"].str.cat(df8["名"])

0    张三
1    李四
2    王五
Name: 姓, dtype: object

In [52]:
df8["姓"].str.cat(df8["名"], sep="-")

0    张-三
1    李-四
2    王-五
Name: 姓, dtype: object

In [53]:
type(df8["姓"].str.cat(df8["名"], sep="-"))

pandas.core.series.Series

In [54]:
df8["姓名"] = df8["姓"].str.cat(df8["名"])
df8

,姓,名,年龄,姓名
0,张,三,30,张三
1,李,四,25,李四
2,王,五,35,王五


In [55]:
df8.drop(["姓", "名"], axis=1,inplace=True)
df8

,年龄,姓名
0,30,张三
1,25,李四
2,35,王五


# 把宽数据转换成长数据-使用melt函数将多列变量转换为一列

In [2]:
import pandas as pd
df9 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example6.csv")
df9

,国家代码,年份,男性年龄组（0-4岁）,男性年龄组（5-14岁）,男性年龄组（15-24岁）,女性年龄组（0-4岁）,女性年龄组（5-14岁）,女性年龄组（15-24岁）
0,CN,2019,100,200,500,80,150,400
1,US,2019,50,150,300,40,100,200
2,JP,2019,30,120,250,20,80,150
3,IN,2019,80,180,400,60,120,300


In [4]:
pd.melt(
    df9,
    id_vars=['国家代码', '年份'],  
    var_name='年龄组',  
    value_name='肺结核病例数'  
)

,国家代码,年份,年龄组,肺结核病例数
0,CN,2019,男性年龄组（0-4岁）,100
1,US,2019,男性年龄组（0-4岁）,50
2,JP,2019,男性年龄组（0-4岁）,30
3,IN,2019,男性年龄组（0-4岁）,80
4,CN,2019,男性年龄组（5-14岁）,200
5,US,2019,男性年龄组（5-14岁）,150
6,JP,2019,男性年龄组（5-14岁）,120
7,IN,2019,男性年龄组（5-14岁）,180
8,CN,2019,男性年龄组（15-24岁）,500
9,US,2019,男性年龄组（15-24岁）,300


melt 的过程就像“融化”数据。它会将你指定的某些列（代表变量）的列名变成一个新列（var_name），同时将这些列中的值变成另一个新列（value_name）。<br>id_vars指定了哪些列是标识符，这些列在转换过程中保持不变<br>
var_name指定了新列的名称，这个新列将用来存放被“融化”的那些列的列名即：未被id_vars选中的列会被融化<br>value_name 指定了另一个新列的名称，这个新列将用来存放被“融化”的那些列中的数值                                                                               

# 对行进行拆分

In [19]:
import pandas as pd
df10 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example7.csv")
df10

,学生姓名,学号,课程列表
0,张三,1,"['数学', '物理']"
1,李四,2,"['英语', '化学', '历史']"
2,王五,3,"['语文', '数学', '英语', '政治']"
3,赵六,4,"['物理', '生物']"


 虽然上面的课程列表变量看起来像是列表，但实际上是字符串<br>
 比如第一行里的课程列表实际为字符串"['数学', '物理']"，而不是列表['数学', '物理']<br>
 所以我们需要先将字符串形式的列表转换为实际的列表对象，这样explode才能对列表进行拆分<br>
 这里我们可以使用Python内置的`eval`函数，它可以把字符串转换成表达式，所以可以帮我们把表示列表的字符串转换成列表本身<br>

In [20]:
df10['课程列表'] = df10['课程列表'].apply(lambda x: eval(x))

In [21]:
df10

,学生姓名,学号,课程列表
0,张三,1,"[数学, 物理]"
1,李四,2,"[英语, 化学, 历史]"
2,王五,3,"[语文, 数学, 英语, 政治]"
3,赵六,4,"[物理, 生物]"


In [22]:
df10.explode("课程列表")

,学生姓名,学号,课程列表
0,张三,1,数学
0,张三,1,物理
1,李四,2,英语
1,李四,2,化学
1,李四,2,历史
2,王五,3,语文
2,王五,3,数学
2,王五,3,英语
2,王五,3,政治
3,赵六,4,物理


# 对行或列进行删除

In [23]:
df11 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example8.csv")
df11

,姓名,考试1,考试2,考试3
0,小陈,85,95,92
1,小李,91,92,94
2,小王,86,81,89
3,小张,79,89,95
4,小赵,96,91,91
5,小周,81,89,92


In [11]:
df11.drop(2)

,姓名,考试1,考试2,考试3
0,小陈,85,95,92
1,小李,91,92,94
3,小张,79,89,95
4,小赵,96,91,91
5,小周,81,89,92


In [12]:
df11.drop("考试2", axis=1)

,姓名,考试1,考试3
0,小陈,85,92
1,小李,91,94
2,小王,86,89
3,小张,79,95
4,小赵,96,91
5,小周,81,92


In [13]:
df11.drop([2, 4])

,姓名,考试1,考试2,考试3
0,小陈,85,95,92
1,小李,91,92,94
3,小张,79,89,95
5,小周,81,89,92


In [14]:
df11.drop(["考试2", "考试3"], axis=1)

,姓名,考试1
0,小陈,85
1,小李,91
2,小王,86
3,小张,79
4,小赵,96
5,小周,81


In [15]:
df11

,姓名,考试1,考试2,考试3
0,小陈,85,95,92
1,小李,91,92,94
2,小王,86,81,89
3,小张,79,89,95
4,小赵,96,91,91
5,小周,81,89,92


In [24]:
df12 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\example8.csv",index_col=0)
df12

,考试1,考试2,考试3
姓名,,,
小陈,85,95,92
小李,91,92,94
小王,86,81,89
小张,79,89,95
小赵,96,91,91
小周,81,89,92


In [25]:
df12.drop(['小王', '小李'], inplace=True)
df12

,考试1,考试2,考试3
姓名,,,
小陈,85,95,92
小张,79,89,95
小赵,96,91,91
小周,81,89,92


# 对整列缺失值进行填充

In [27]:
import pandas as pd
import numpy as np

In [29]:
df13 = pd.DataFrame({
    '日期': ['2005-01-01', '2005-01-02', '2005-01-03', '2005-01-03'],
    '销售额': [1000, 1500, 800, 1200],
    '销售人员': ['李华', '王磊', '刘娜', '张洋'],
    '地址': ['苏州', '郑州', '南京', '西安']
})
df13 = df13.rename(index={
    0: '001',
    1: '002',
    2: '003',
    3: '004'
})
df13['国家'] = np.nan
df13

,日期,销售额,销售人员,地址,国家
001,2005-01-01,1000,李华,苏州,NaN
002,2005-01-02,1500,王磊,郑州,NaN
003,2005-01-03,800,刘娜,南京,NaN
004,2005-01-03,1200,张洋,西安,NaN


In [30]:
df13["国家"] = "中国"
df13

,日期,销售额,销售人员,地址,国家
001,2005-01-01,1000,李华,苏州,中国
002,2005-01-02,1500,王磊,郑州,中国
003,2005-01-03,800,刘娜,南京,中国
004,2005-01-03,1200,张洋,西安,中国


# 对某个缺失值进行填充

In [32]:
df14 = pd.DataFrame({
    '日期': ['2005-01-01', '2005-01-02', '2005-01-03', '2005-01-03'],
    '销售额': [1000, 1500, np.nan, 1200],
    '销售人员': ['李华', '王磊', '刘娜', '张洋'],
    '地址': ['苏州', '郑州', '南京', '西安']
})
df14 = df14.rename(index={
    0: '001',
    1: '002',
    2: '003',
    3: '004'
})
df14

,日期,销售额,销售人员,地址
001,2005-01-01,1000.0,李华,苏州
002,2005-01-02,1500.0,王磊,郑州
003,2005-01-03,NaN,刘娜,南京
004,2005-01-03,1200.0,张洋,西安


In [33]:
df14.loc["003", "销售额"] = 800
df14

,日期,销售额,销售人员,地址
001,2005-01-01,1000.0,李华,苏州
002,2005-01-02,1500.0,王磊,郑州
003,2005-01-03,800.0,刘娜,南京
004,2005-01-03,1200.0,张洋,西安


In [34]:
df14.iloc[0, 1] = 80000000
df14

,日期,销售额,销售人员,地址
001,2005-01-01,80000000.0,李华,苏州
002,2005-01-02,1500.0,王磊,郑州
003,2005-01-03,800.0,刘娜,南京
004,2005-01-03,1200.0,张洋,西安


# 对部分缺失值进行填充

In [35]:
df15 = pd.DataFrame({
    '日期': ['2005-01-01', '2005-01-02', np.nan, np.nan],
    '销售额': [1000, 1500, 800, 1200],
    '销售人员': ['李华', '王磊', '刘娜', '张洋'],
    '地址': ['苏州', '郑州', '南京', '西安']
})
df15 = df15.rename(index={
    0: '001',
    1: '002',
    2: '003',
    3: '004'
})
df15

,日期,销售额,销售人员,地址
001,2005-01-01,1000,李华,苏州
002,2005-01-02,1500,王磊,郑州
003,NaN,800,刘娜,南京
004,NaN,1200,张洋,西安


In [36]:
df15.loc["003":"004", "日期"] = "2005-01-03"
df15

,日期,销售额,销售人员,地址
001,2005-01-01,1000,李华,苏州
002,2005-01-02,1500,王磊,郑州
003,2005-01-03,800,刘娜,南京
004,2005-01-03,1200,张洋,西安


In [37]:
df15.iloc[0:3, 0] = "2005-01-03"
df15

,日期,销售额,销售人员,地址
001,2005-01-03,1000,李华,苏州
002,2005-01-03,1500,王磊,郑州
003,2005-01-03,800,刘娜,南京
004,2005-01-03,1200,张洋,西安


In [41]:
df15.iloc[0:3, 1:3] = ["888","yancy"]
df15

,日期,销售额,销售人员,地址
001,2005-01-03,888,yancy,苏州
002,2005-01-03,888,yancy,郑州
003,2005-01-03,888,yancy,南京
004,2005-01-03,1200,张洋,西安


# 自动找到缺失值进行填充

In [44]:
df16 = pd.DataFrame({'A': [1, 2, np.nan, 4],
                   'B': [5, np.nan, 7, np.nan],
                   'C': [8, 9, 10, 11]})
df16

,A,B,C
0,1.0,5.0,8
1,2.0,NaN,9
2,NaN,7.0,10
3,4.0,NaN,11


In [45]:
df16["B"].fillna(0)

0    5.0
1    0.0
2    7.0
3    0.0
Name: B, dtype: float64

In [46]:
df16["B"].fillna(df16["B"].mean())

0    5.0
1    6.0
2    7.0
3    6.0
Name: B, dtype: float64

In [47]:
df16.fillna(0)

,A,B,C
0,1.0,5.0,8
1,2.0,0.0,9
2,0.0,7.0,10
3,4.0,0.0,11


In [48]:
df16.fillna({'A': 0, 'B': 10, 'C': 20})

,A,B,C
0,1.0,5.0,8
1,2.0,10.0,9
2,0.0,7.0,10
3,4.0,10.0,11


In [49]:
df16.fillna({'A': df16["A"].mean(), 'B': df16["B"].mean(), 'C': df16["C"].mean()}, inplace=True)
df16

,A,B,C
0,1.000000,5.0,8
1,2.000000,6.0,9
2,2.333333,7.0,10
3,4.000000,6.0,11


*在求平均值的时候pandas会自动忽略值为nan的那一行*

# 删除存在缺失值的行/列

In [50]:
df17 = pd.DataFrame({
    '姓名': ['John', 'Alice', 'Bob', 'Mary'],
    '年龄': [25, 30, np.nan, 40],
    '工资': [50000, np.nan, 70000, 60000],
    '性别': ['M', 'F', 'M', 'F']
})
df17

,姓名,年龄,工资,性别
0,John,25.0,50000.0,M
1,Alice,30.0,NaN,F
2,Bob,NaN,70000.0,M
3,Mary,40.0,60000.0,F


In [51]:
df17.dropna()

,姓名,年龄,工资,性别
0,John,25.0,50000.0,M
3,Mary,40.0,60000.0,F


In [52]:
df17.dropna(subset=["工资"])

,姓名,年龄,工资,性别
0,John,25.0,50000.0,M
2,Bob,NaN,70000.0,M
3,Mary,40.0,60000.0,F


In [53]:
df17.dropna(axis=1)

,姓名,性别
0,John,M
1,Alice,F
2,Bob,M
3,Mary,F


In [54]:
df17.dropna(axis=1, subset=[1, 2])

,姓名,性别
0,John,M
1,Alice,F
2,Bob,M
3,Mary,F


# 删除重复数据

In [55]:
df18 = pd.DataFrame({
    '姓名': ['John', 'Alice', 'Bob', 'Alice', 'John'],
    '年龄': [25, 30, 35, 30, 40],
    '性别': ['M', 'F', 'M', 'F', 'M']
})
df18

,姓名,年龄,性别
0,John,25,M
1,Alice,30,F
2,Bob,35,M
3,Alice,30,F
4,John,40,M


In [57]:
df18["姓名"].drop_duplicates()

0     John
1    Alice
2      Bob
Name: 姓名, dtype: object

**因为df由Series构成故很多方法对df与s都共享**

In [58]:
print(df18)
df18.drop_duplicates()

      姓名  年龄 性别
0   John  25  M
1  Alice  30  F
2    Bob  35  M
3  Alice  30  F
4   John  40  M


,姓名,年龄,性别
0,John,25,M
1,Alice,30,F
2,Bob,35,M
4,John,40,M


In [60]:
print(df18)
df18.drop_duplicates(subset=['姓名', '性别'])

      姓名  年龄 性别
0   John  25  M
1  Alice  30  F
2    Bob  35  M
3  Alice  30  F
4   John  40  M


,姓名,年龄,性别
0,John,25,M
1,Alice,30,F
2,Bob,35,M


In [62]:
print(df18)
df18.drop_duplicates(subset=['姓名', '性别'], keep='last',inplace=True)
df18

      姓名  年龄 性别
0   John  25  M
1  Alice  30  F
2    Bob  35  M
3  Alice  30  F
4   John  40  M


,姓名,年龄,性别
2,Bob,35,M
3,Alice,30,F
4,John,40,M


keep='last' 时，只保留每组重复记录中的最后一行，删除其他重复行

# 对值进行替换

In [63]:
data = {'姓名': ['小明', '小红', '小张', '小李'],
        '家乡': ['北京', '上海', '广州', '深圳'],
        '学校': ['北京大学', '清华大学', '华南理工', '清华']}
df19 = pd.DataFrame(data)
df19

,姓名,家乡,学校
0,小明,北京,北京大学
1,小红,上海,清华大学
2,小张,广州,华南理工
3,小李,深圳,清华


In [64]:
df19["学校"].replace("清华", "清华大学")

0    北京大学
1    清华大学
2    华南理工
3    清华大学
Name: 学校, dtype: object

In [65]:
df19.replace("清华", "清华大学")

,姓名,家乡,学校
0,小明,北京,北京大学
1,小红,上海,清华大学
2,小张,广州,华南理工
3,小李,深圳,清华大学


In [67]:
df19["学校"].replace(["清华", "五道口职业技术学院", "Tsinghua University"], "清华大学")

0    北京大学
1    清华大学
2    华南理工
3    清华大学
Name: 学校, dtype: object

In [68]:
df19.replace(["清华", "五道口职业技术学院", "Tsinghua University"], "清华大学")

,姓名,家乡,学校
0,小明,北京,北京大学
1,小红,上海,清华大学
2,小张,广州,华南理工
3,小李,深圳,清华大学


In [69]:
replace_dict = {'华南理工': '华南理工大学',
                '清华': '清华大学',
                '北大': '北京大学',
                '中大': '中山大学',
                '北京': '南京'}
df19.replace(replace_dict)

,姓名,家乡,学校
0,小明,南京,北京大学
1,小红,上海,清华大学
2,小张,广州,华南理工大学
3,小李,深圳,清华大学


# 对值的类型进行转换

**对于有电话号码属性的df中若该属性下的值被当作int/float那么调用describe()这一列的信息显示将毫无意义，<br>当满意度属性下的数据类型的为str的是/否而不是bool的True/False那么describe()将显示不出该满意度列下的有用信息,所以这时候需要对值进行类型转换**

In [71]:
s1 = pd.Series([1, 2, 3])
print(s1)
s1.astype("float")

0    1
1    2
2    3
dtype: int64


0    1.0
1    2.0
2    3.0
dtype: float64

In [72]:
s1

0    1
1    2
2    3
dtype: int64

*astype依旧不改变原Series*

In [73]:
print(type('4'))
print(type('1'))
print(type(True))
print(type(3.0))
type([1,'a'])

<class 'str'>
<class 'str'>
<class 'bool'>
<class 'float'>


list

In [74]:
print(s1)
s1.astype(str)

0    1
1    2
2    3
dtype: int64


0    1
1    2
2    3
dtype: object

我们可以直接把object类型当作表示字符类型或整数类型

In [75]:
s2 = pd.Series(["红色", "红色", "橙色", "蓝色"])
s2.astype("category")

0    红色
1    红色
2    橙色
3    蓝色
dtype: category
Categories (3, object): ['橙色', '红色', '蓝色']

category是分类数据，数据分为分类数据和数值数据，分类数据是指包含有限数量的不同类别数据如性别只有两种，奖牌只有金银铜，数值数据是某个具体数据种类不受限制比如身高有无数可能的值，这类数据，当数据的种类有限时候比如：颜色，国家，学历那么建议把这些数据astype转为catagory数据类型节省空间利于数据可视化

# 保存清洗后的数据

*将清洗后的数据写入新的csv文件*

In [76]:
import pandas as pd
import numpy as np
df111 = pd.DataFrame({
    '日期': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04'],
    '销售额': [1000, 1500, 800, 1200],
    '销售人员': ['Alice', 'Bob', 'Charlie', 'David'],
    '城市': ['New York', 'San Francisco', 'New York', 'San Francisco']
})
df111 = df111.rename(index={
    0: '38H9',
    1: 'W9F1',
    2: 'KD82',
    3: '004U'
})
df111

,日期,销售额,销售人员,城市
38H9,2022-01-01,1000,Alice,New York
W9F1,2022-01-02,1500,Bob,San Francisco
KD82,2022-01-03,800,Charlie,New York
004U,2022-01-04,1200,David,San Francisco


.to_csv()将df以文件形式保存，若原文件存在则把之前内容删除并写入，原文件不存在就创建一个并写入

In [78]:
import os
print(os.getcwd())


C:\Users\杨加春\data_analysis


In [77]:
df111.to_csv("cleaned_sales_data.csv")

*该文件将会在路径  C:\Users\杨加春\data_analysis 这个位置生成一个文件*

In [79]:
cleaned_df = pd.read_csv("cleaned_sales_data.csv")
cleaned_df

,Unnamed: 0,日期,销售额,销售人员,城市
0,38H9,2022-01-01,1000,Alice,New York
1,W9F1,2022-01-02,1500,Bob,San Francisco
2,KD82,2022-01-03,800,Charlie,New York
3,004U,2022-01-04,1200,David,San Francisco


**你会发现当一个df在经过写入与再读取后会与一开始读取的df不同！<br>这是因为你一开始写入时候原封不动写入，当再读取的时候会默认把你原df中的标签索引解读为没有属性名的列第一行解读为列名**

**所以当你想写入df时候最好不要让df像开头那样的结构**

## 保存方式一

In [80]:
cleaned_df.rename(columns={"Unnamed: 0": "销售ID"}, inplace=True)
cleaned_df

,销售ID,日期,销售额,销售人员,城市
0,38H9,2022-01-01,1000,Alice,New York
1,W9F1,2022-01-02,1500,Bob,San Francisco
2,KD82,2022-01-03,800,Charlie,New York
3,004U,2022-01-04,1200,David,San Francisco


In [81]:
cleaned_df.set_index("销售ID", inplace=True)
cleaned_df

,日期,销售额,销售人员,城市
销售ID,,,,
38H9,2022-01-01,1000,Alice,New York
W9F1,2022-01-02,1500,Bob,San Francisco
KD82,2022-01-03,800,Charlie,New York
004U,2022-01-04,1200,David,San Francisco


In [88]:
df111

,日期,销售额,销售人员,城市
38H9,2022-01-01,1000,Alice,New York
W9F1,2022-01-02,1500,Bob,San Francisco
KD82,2022-01-03,800,Charlie,New York
004U,2022-01-04,1200,David,San Francisco


*把清理后的df写入文件时候可以像开头那样写入默认的地址，也可以指定地址：用新建一个文件/选择一个无用文件的文件地址*

In [84]:
cleaned_df.to_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\cleaned_sales_data2.csv.txt")

In [85]:
df222 = pd.read_csv(r"F:\BaiduNetdiskDownload\数据分析课程操作的文件\04数据评估与清洗篇\cleaned_sales_data2.csv.txt")
df222

,销售ID,日期,销售额,销售人员,城市
0,38H9,2022-01-01,1000,Alice,New York
1,W9F1,2022-01-02,1500,Bob,San Francisco
2,KD82,2022-01-03,800,Charlie,New York
3,004U,2022-01-04,1200,David,San Francisco


## 保存方式二

In [89]:
df111.to_csv("cleaned_sales_data3.csv", index=False)
df111

,日期,销售额,销售人员,城市
38H9,2022-01-01,1000,Alice,New York
W9F1,2022-01-02,1500,Bob,San Francisco
KD82,2022-01-03,800,Charlie,New York
004U,2022-01-04,1200,David,San Francisco


In [87]:
cleaned_df_without_index = pd.read_csv("cleaned_sales_data3.csv")
cleaned_df_without_index

,日期,销售额,销售人员,城市
0,2022-01-01,1000,Alice,New York
1,2022-01-02,1500,Bob,San Francisco
2,2022-01-03,800,Charlie,New York
3,2022-01-04,1200,David,San Francisco
